In [111]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import mne
import neurokit2 as nk

load_dotenv()


pwd = os.getenv('TUH_PASSWORD')
user = os.getenv('TUH_USER')
path_data = os.getenv('PATH_DATASET')


PATH = os.path.abspath('../../')
PATH

'/home/davi/Github/doutorado_ppgee_v2'

## Parâmetros iniciais

In [112]:
df1 = pd.read_csv(PATH+'/data/Generalized_seizures_dataset.csv') # Seizure

df2 = pd.read_csv(PATH + '/data/data_train.csv') # Normal

In [113]:
fs = 256 # freq amostral

## Funções básicas

In [133]:
# Carregando .edf

def load_edf(file):
    data = mne.io.read_raw_edf(path_data + '/' + file)
    info = data.info
    return {"info":info,"raw":data}

# Canal de ECG puro 
def raw_to_dataframe(signal,channels):
    df = signal.to_data_frame()
    limit_start = fs*10 # limitando 10 segundos iniciais
    limit_end = len(ecg_data1)-(fs*10) # limitando 10 segundos finais
    return df[channels].iloc[limit_start:limit_end]*(-1)

# Identificar canais de ecg
def get_ecg_channels(info):
    ch_ecg = [ch for ch in info.ch_names if 'EKG' in ch]
    return ch_ecg


# Filtrando ECG
# Recuperando batimentos cardíacos
def ecg_process(signal,fs):
    ecg_clean = nk.ecg_clean(signal, sampling_rate=fs, method="hamilton2002")
    return nk.ecg_segment(ecg_clean, rpeaks=None, sampling_rate=256, show=False)

## Pipeline de processamento

In [115]:
ecg1_path = df1[df1.freq==fs].path.iloc[0].replace('tse', 'edf')
ecg2_path = df2[df2.freq==fs].path.iloc[0].replace('tse', 'edf')

ecg1_infos = load_edf(ecg1_path)
ecg2_infos = load_edf(ecg2_path)

ecg1_ch = get_ecg_channels(ecg1_infos['info'])
ecg2_ch = get_ecg_channels(ecg2_infos['info'])

ecg_data1 = raw_to_dataframe(ecg1_infos['raw'],ecg1_ch)
ecg_data2 = raw_to_dataframe(ecg2_infos['raw'],ecg2_ch)

Extracting EDF parameters from /media/davi/2526467b-9f07-4ce5-9425-6d2b458567b7/home/davi/www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_seizure/v1.5.2/edf/train/01_tcp_ar/044/00004456/s015_2014_06_19/00004456_s015_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /media/davi/2526467b-9f07-4ce5-9425-6d2b458567b7/home/davi/www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_seizure/v1.5.2/edf/train/01_tcp_ar/004/00000458/s011_2014_06_24/00000458_s011_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [116]:
# Intervalo de de crise
star_seizure = int(df1[df1.freq==fs].iloc[0]['inicio'])*fs
stop_seizure = int(df1[df1.freq==fs].iloc[0]['fim'])*fs

normal_interval = stop_seizure - star_seizure


In [117]:
seizure_segment = ecg_data1.reset_index(drop=True).iloc[star_seizure:stop_seizure]
normal_segment = ecg_data2.reset_index(drop=True).iloc[:normal_interval]

In [121]:
result1 = ecg_process(seizure_segment,fs)
result2 = ecg_process(normal_segment,fs)